In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Path hack.
import sys, os
from os.path import join, basename, dirname, exists
sys.path.insert(0, os.path.abspath('..'))
from scripts.data_utils.generators import SeismicGenerator, SequencePathGenerator
from scripts.data_utils.grid_utils import blockshaped, unblockshaped, plot_embeddings
from cpc.train_model_seimsic import network_encoder, network_autoregressive, network_prediction, CPCLayer, network_cpc, train_model

# declare parent dir name 
dirname = sys.path[0] # parent directory

"""
Representation Learning with Contrastive Predictive Coding
The goal of unsupervised representation learning is to capture semantic information about the world,
recognizing patterns in the data without using annotations. 
This paper presents a new method called Contrastive Predictive Coding (CPC) that can do so across multiple applications. 
The main ideas of the paper are:
* Contrastive: it is trained using a contrastive approach, that is,
  the main model has to discern between *right* and *wrong* data sequences.
* Predictive: the model has to predict future patterns given the current context.
* Coding: the model performs this prediction in a latent space, transforming code vectors 
  into other code vectors (in contrast with predicting high-dimensional data directly).

CPC has to predict the next item in a sequence using only an embedded representation of the data, 
xrovided by an encoder. In order to solve the task, this encoder has to learn a meaningful representation 
of the data space. After training, this encoder can be used for other downstream tasks like supervised classification.




The cpc article descibes the following pattern:

Pre-processing:
-(own) from a random iline/xline/tline > 256*256 we extract pathces of 256x256 using sklearn image.extract_patches_2d
-from a 256x256 image we extract a 7x7 grid of 64x64 crops
with 32 pixels overlap. 

-(TODO)Simple data augmentation proved helpful on both the 256x256 images and the
64x64 crops. 
-(DONE)The 256x256 images are randomly cropped from a 300x300 image
- horizontally flipped with a probability of 50% and converted to greyscale. 
-For each of the 64x64 crops we randomly take
a 60x60 subcrop and pad them back to a 64x64 image.
-MNIST example uses RGB channels. Seimsic is grayscale. Would it help to pick and generate x attributes as channels
Encoder:
-(DONE)Each crop is then encoded by the ResNet-v2-101 encoder
- (DONE)We use the outputs from the third residual
block, and spatially mean-pool to get a single 1024-d vector per 64x64 patch. This results in a
7x7x1024 tensor.
"""

# change the patch_size and stride to get different girds. if stride is less than patch_size--> we have overlap on each grid cell
image_size = 256
patch_size = 64
stride=32


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# train model 
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
print(timestr)
train_model(
        epochs=5,
        batch_size=1,
        output_dir='models/cpc/seismic_64x64_'+ timestr,
        code_size=1024,
        lr=1e-3,
        terms=3,
        predict_terms=4,
        patch_size=64,
    )
print("Done training")

In [ ]:
#TODO NEED TO SPECIDY steps_per_epoch